In [16]:
%load_ext autoreload
%autoreload 2

from transformer_lens import HookedTransformer
import torch as t
from huggingface_hub import login
from contextlib import contextmanager
import json
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc
from IPython.display import display, HTML
import html as html_escaper

from utils import *

# login("")
t.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
model_base = HookedTransformer.from_pretrained_no_processing("gemma-2-2b", device="cpu")
model_chat = HookedTransformer.from_pretrained_no_processing("gemma-2-2b-it", device="cuda")
model_hs = HookedTransformer.from_pretrained_no_processing("gemma-2-2b-it", device="cuda")
clear_mem()

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [32]:
MAX_SEQ_LEN = 300
NUM_LINES = 2000
with open("2b_it_generations.jsonl", "r") as f:
    data = [json.loads(line)['conversation'] for i, line in enumerate(f) if i < NUM_LINES]
toks = model_base.to_tokens(data, prepend_bos=False)[:, :MAX_SEQ_LEN] 

In [34]:
clear_mem()

layer, head = 15, 5
kl_hs_chat, kl_ablated_chat = get_kl(model_base, model_hs, model_chat, toks[:20], layer, head, batch_size=16)

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
top_k_kls, top_k_indices = matrix_topk(kl_hs_chat, k=10)
for (batch_idx, seq_idx) in top_k_indices:
    highlight_kl(model_base, model_hs, model_chat, toks[batch_idx, :seq_idx+30], layer, head, cscale=0.02)
    print("--"*100)

High KL: 0.0089 at token 'rax'\n


,Token,p_HS,p_B
0,$,0.063,0.071
1,,0.087,0.097
2,r,0.197,0.216
3,eax,0.144,0.127
4,rax,0.194,0.153


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0089 at token 'rax'\n


,Token,p_HS,p_B
0,$,0.063,0.071
1,,0.087,0.097
2,r,0.197,0.216
3,eax,0.144,0.127
4,rax,0.194,0.153


...

,Token,p_HS,p_B
0,,0.226,0.250
1,,0.050,0.054
2,*,0.684,0.652


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0089 at token 'rax'\n


,Token,p_HS,p_B
0,$,0.063,0.071
1,,0.087,0.097
2,r,0.197,0.216
3,eax,0.144,0.127
4,rax,0.194,0.153


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
